In [1]:
import pandas as pd
import numpy as np

### Perimeter

In [2]:
tconst = pd.read_parquet("../data/edited_data/datamart.db/tconst")

In [3]:
title_ratings = pd.read_parquet("../data/edited_data/datamart.db/title_ratings")

In [17]:
avg_rating = pd.read_parquet("../data/edited_data/intermediate.db/avg_rating")
genres = pd.read_parquet("../data/edited_data/intermediate.db/genres")
#primary_profession = pd.read_parquet("../data/edited_data/intermediate.db/primary_profession")
primary_title = pd.read_parquet("../data/edited_data/intermediate.db/primary_title")
title_budget = pd.read_parquet("../data/edited_data/intermediate.db/title_budget")

### modeling.db

In [13]:
title_budget.head()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,titleBasicsFlag
title,,,,,,
10000bc,2008-03-07,"10,000 B.C.",105000000.0,94784201.0,269065678.0,True
102dalmatians,2000-11-22,102 Dalmatians,85000000.0,66941559.0,66941559.0,True
10cloverfieldlane,2016-01-04,10 Cloverfield Lane,5000000.0,72082999.0,108286422.0,True
10daysinamadhouse,2015-11-11,10 Days in a Madhouse,12000000.0,14616.0,14616.0,True
10thingsihateaboutyou,1999-03-31,10 Things I Hate About You,13000000.0,38177966.0,60413950.0,True


In [21]:
data = tconst.set_index("tconst").join(genres).join(avg_rating).join(title_budget)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 669916 entries, tt0000038 to tt9916764
Data columns (total 37 columns):
genre_action         652487 non-null float64
genre_adult          652487 non-null float64
genre_adventure      652487 non-null float64
genre_animation      652487 non-null float64
genre_biography      652487 non-null float64
genre_comedy         652487 non-null float64
genre_crime          652487 non-null float64
genre_documentary    652487 non-null float64
genre_drama          652487 non-null float64
genre_family         652487 non-null float64
genre_fantasy        652487 non-null float64
genre_film-noir      652487 non-null float64
genre_game-show      652487 non-null float64
genre_history        652487 non-null float64
genre_horror         652487 non-null float64
genre_music          652487 non-null float64
genre_musical        652487 non-null float64
genre_mystery        652487 non-null float64
genre_news           652487 non-null float64
genre_reality-tv     652487 

In [22]:
data.head()

,genre_action,genre_adult,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,genre_documentary,genre_drama,genre_family,...,genre_western,averageRating,numVotes,title,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,titleBasicsFlag
tconst,,,,,,,,,,,,,,,,,,,,,
tt0000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
tt0000417,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
tt0001440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
tt0001563,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
tt0001889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
